In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
library(googlesheets)
gs_auth(token = "/home/users/ytanigaw/.googlesheets_token.rds")

file <- 'https://docs.google.com/spreadsheets/d/1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A'
GBE_names_df <- file %>% gs_url() %>% gs_read(ws = 'GBE_names')


Auto-refreshing stale OAuth token.
Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A
Sheet successfully identified: "GBE_names"
Accessing worksheet titled 'GBE_names'.
Parsed with column specification:
cols(
  GBE_category = col_character(),
  GBE_ID = col_character(),
  GBE_N = col_double(),
  GBE_NAME = col_character(),
  GBE_short_name = col_character(),
  GBE_short_name_len = col_double()
)


In [3]:
res_dir <- '@@@@@@/projects/biomarkers/phewas/v2_imp_nc'
phewas_hits <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.tsv')
phewas_ld   <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ldmap.tsv')
annot_file  <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ldmap.annot.tsv')


In [4]:
phewas_hits_ld <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.tsv')


#### we use FinnGen R2 sumstats to map rsid to genomic position in GRCh38

In [5]:
finngen_r2_sumstats <- '@@@@@@@@/users/ytanigaw/20200114_FinnGen_R2/summary_stats/finngen_r2_COPD_COMORB.gz'


## read files

In [6]:
ld_df <- fread(phewas_ld, sep='\t') %>% rename('CHROM' = '#CHROM')
phewas_df <- fread(phewas_hits, sep='\t')
annot_df <- fread(annot_file, sep='\t') %>% rename('CHROM' = '#CHROM')


In [7]:
finngen_df <- fread(
    cmd=paste('zcat', finngen_r2_sumstats), 
    select=c('#chrom', 'pos', 'ref', 'alt', 'rsids')
) %>%
rename('chrom' = '#chrom') %>%
mutate(
    finngen_var_id = paste(chrom, pos, ref, alt, sep='-')
) %>%
select(rsids, finngen_var_id)


## join tables

In [8]:
df <- phewas_df %>%
left_join(
    GBE_names_df %>% select(GBE_ID, GBE_short_name), 
    by='GBE_ID'
) %>%
left_join(
    ld_df %>% 
    mutate(LD_ID_dup = LD_ID) %>%
    separate(LD_ID_dup, c(NA, NA, 'LD_REF', 'LD_ALT'), sep=':'),
    by=c('CHROM', 'POS', 'ID')
) %>%
left_join(
    annot_df %>% 
    select(CHROM, POS, REF, ALT, ID, Consequence, Gene, Gene_symbol) %>%
    rename('LD_rsID' = 'ID'),
    by=c('LD_CHROM' = 'CHROM', 'LD_POS' = 'POS', 'LD_REF' = 'REF', 'LD_ALT' = 'ALT')
) %>%
select(-LD_REF, -LD_ALT) %>%
arrange(CHROM, POS, LD_CHROM, LD_POS)


In [9]:
df %>% dim()

[1] 2297   18

### check if we recovered the variant IDs

In [10]:
df %>% filter(is.na(LD_rsID)) %>% dim()

[1] 17 18

In [11]:
df %>% filter(is.na(LD_rsID)) %>% 
select(LD_ID) %>% unique()

,LD_ID
,<chr>
1,1:154343714:CAA:C
2,1:161599643:T:C
3,2:204792071:A:T
7,6:161068064:G:C
9,15:79026674:G:A
10,15:79054932:G:C
11,15:79056769:T:C
12,15:79059449:G:A
13,15:79059703:T:C


#### using gnomAD, fill in the missing rsIDs

In [12]:
rsid_map_manual <- list(
    '1:154343714:CAA:C' = 'rs78080804',
    '1:161599643:T:C'   = 'rs147574249',
    '2:204792071:A:T'   = 'rs6435212',
    '6:161068064:G:C'   = 'rs6905422',
    '15:79026674:G:A'   = 'rs4886586',    
    '15:79054932:G:C'   = 'rs11072801',
    '15:79056769:T:C'   = 'rs1809420',
    '15:79059449:G:A'   = 'rs1810165',
    '15:79059703:T:C'   = 'rs1809424',
    '19:49213853:AT:A'  = 'rs1244201130'
)


In [13]:
for(var_id in names(rsid_map_manual)){
    df[ df[['LD_ID']] == var_id ,  ][['LD_rsID']] <- rsid_map_manual[[ var_id ]]
}


In [14]:
df %>% filter(is.na(LD_rsID)) %>% 
select(LD_ID) %>% unique()

LD_ID
<chr>


### merge with Finngen variant IDs

In [15]:
df_with_finngen <- df %>%
left_join(
    finngen_df,
    by=c('LD_rsID' = 'rsids')
)


In [16]:
df_with_finngen %>% filter(is.na(finngen_var_id)) %>% dim()

[1] 516  19

In [17]:
df_no_finngen_match <- df_with_finngen %>% 
select(CHROM, POS, ID, finngen_var_id) %>% 
group_by(CHROM, POS, ID) %>%
summarise(n_finngen_non_NAs = sum(! is.na(finngen_var_id)) ) %>%
filter(n_finngen_non_NAs == 0) %>%
select(-n_finngen_non_NAs) %>%
arrange(CHROM, POS) %>% unique()

In [18]:
df_no_finngen_match %>% dim()

[1] 33  3

##### 516 variants (corresponding to 33 variants before the LD mapping) have no entry in the FinnGen 

In [19]:
df_with_finngen %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(phewas_hits_ld, sep='\t')


In [20]:
phewas_hits_ld

[1] "@@@@@@/projects/biomarkers/phewas/v2_imp_nc/ukb24983_imp_v3.nc.phewas.hits.ld.tsv"

In [21]:
df_with_finngen %>% colnames()

[1] "CHROM"          "POS"            "ID"             "OBS_CT"        
 [5] "OR"             "LOG(OR)_SE"     "Z_STAT"         "P"             
 [9] "GBE_ID"         "GBE_short_name" "LD_CHROM"       "LD_POS"        
[13] "LD_ID"          "LD_R2"          "LD_rsID"        "Consequence"   
[17] "Gene"           "Gene_symbol"    "finngen_var_id"